In [ ]:
# для работы с документами
!pip install python-docx
# классификатор
!pip install catboost

     |████████████████████████████████| 5.6 MB 27.6 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=05631a09171cc4f1a11b44a7e40574010aa308e258d2c0055384d489ef98df6f
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
     |████████████████████████████████| 67.4 MB 48 kB/s 


In [ ]:
!gdown --id 1zJnmnqJ7uIJz2FNjWYgXbihtx6vT-DH5
!7z x DataSet_Razmetra.7z -oDataSet_Razmetra

Downloading...
From: https://drive.google.com/uc?id=1zJnmnqJ7uIJz2FNjWYgXbihtx6vT-DH5
To: /content/DataSet_Razmetra.7z
1.34GB [00:13, 99.4MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1335264230 bytes (1274 MiB)

Extracting archive: DataSet_Razmetra.7z
--
Path = DataSet_Razmetra.7z
Type = 7z
Physical Size = 1335264230
Headers Size = 463035
Method = LZMA2:20
Solid = +
Blocks = 14

  0%      0% 3669           0% 7797           0% 10933 - Алтайский край/3_2/BA53E . ition_1/Edition_Text.docx                                                                              

In [ ]:
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm

from docx import Document
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier

In [ ]:
file_names = glob.glob('DataSet_Razmetra/**/**/**/**/Edition_Text.docx')

In [ ]:
res = []
for f_name in file_names:
    res.append(f_name.split('/')[2])

pd.Series(res).value_counts()

4_1    1000
3_3     909
3_7     849
3_1     801
3_9     560
3_5     482
4_2     289
4_3     212
3_2     191
3_6      63
3_4      40
3_8       4
dtype: int64

In [ ]:
# смотрим метрики отдельно для каждого коррупциогенного фактор
# датасет для обучения формируем из размеченных файлов, в которых обнаружен коррупциогенный фактор
# и исправленных файлов
# датасет разбиваем на трейн и валидацию
# получаем TF-IDF из текстов
# обучаем бустинг на этих векторах
# смотрим метрики

for corr_fac in list(pd.Series(res).unique()):
    file_names = glob.glob('DataSet_Razmetra/**/**/**/**/Edition_Text.docx')

    parags = []
    labels = []

    highlighted_cnt = 0

    for f_name in tqdm(file_names):
        if f_name.split('/')[2]==corr_fac:
            try:
                doc = Document(f_name)

                # проверяем, что текст размечен
                for paragraph in doc.paragraphs:
                    is_highlighted_text = False

                    for run in paragraph.runs:
                        if not (run.font.highlight_color == None):
                            is_highlighted_text = True
                            break

                    if is_highlighted_text:
                        break

                # если текст размечен - разбираем на абзацы
                if is_highlighted_text:
                    highlighted_cnt = highlighted_cnt + 1

                    for paragraph in doc.paragraphs:
                        if paragraph.text != '': 
                            is_highlighted_parag = False
                            for run in paragraph.runs:
                                if not (run.font.highlight_color == None):
                                    is_highlighted_parag = True
                                    break
                            if is_highlighted_parag:
                                parags.append(paragraph.text)
                                labels.append(f_name.split('/')[2])
            except:
                pass

    print('Найдено', len(file_names), 'файлов, из них', highlighted_cnt, 'с разметкой')

    file_names = glob.glob('DataSet_Razmetra/**/**/**/**/NC_Edition_Text.docx')

    highlighted_cnt = 0

    for f_name in tqdm(file_names):
        if f_name.split('/')[2]==corr_fac:
            try:
                doc = Document(f_name)

                # проверяем, что текст размечен
                for paragraph in doc.paragraphs:
                    is_highlighted_text = False

                    for run in paragraph.runs:
                        if not (run.font.highlight_color == None):
                            is_highlighted_text = True
                            break

                    if is_highlighted_text:
                        break

                # если текст размечен - разбираем на абзацы
                if is_highlighted_text:
                    highlighted_cnt = highlighted_cnt + 1

                    for paragraph in doc.paragraphs:
                        if paragraph.text != '': 
                            is_highlighted_parag = False
                            for run in paragraph.runs:
                                if not (run.font.highlight_color == None):
                                    is_highlighted_parag = True
                                    break
                            if is_highlighted_parag:
                                parags.append(paragraph.text)
                                labels.append('None')

                # docs.append('\n'.join([p.text for p in doc.paragraphs]))
                # labels.append(file_names[i].split('/')[2])
            except:
                pass

    print('Найдено', len(file_names), 'файлов, из них', highlighted_cnt, 'с разметкой')
    print(len(parags), len(labels))
    
    parags_1 = parags[:len(parags)//2]
    parags_2 = parags[len(parags)//2:]
    labels_1 = labels[:len(labels)//2]
    labels_2 = labels[len(labels)//2:]

    train_x = parags_1[:int(0.75*len(parags_1))] + parags_2[:int(0.75*len(parags_2))]
    train_y = labels_1[:int(0.75*len(labels_1))] + labels_2[:int(0.75*len(labels_2))]
    test_x = parags_1[int(0.75*len(parags_1)):] + parags_2[int(0.75*len(parags_2)):]
    test_y = labels_1[int(0.75*len(labels_1)):] + labels_2[int(0.75*len(labels_2)):]
    len(train_x), len(train_y), len(test_x), len(test_y)

    data_train = pd.DataFrame({'parag': train_x,
                              'label': train_y}).drop_duplicates()

    data_test = pd.DataFrame({'parag': test_x,
                              'label': test_y}).drop_duplicates()

    vectorizer = TfidfVectorizer(min_df=5, max_df=100000, ngram_range=(1,5))
    features_train = vectorizer.fit_transform(data_train.parag)
    print(features_train.shape)

    features_test = vectorizer.transform(data_test.parag)

    clf = CatBoostClassifier(n_estimators=100,
                            eval_metric='F1')
    clf.fit(features_train, data_train.label, 
            eval_set=(features_test, data_test.label))
    y_test_pred = clf.predict(features_test)

    print(corr_fac)
    print(classification_report(data_test.label, y_test_pred))

  0%|          | 0/5400 [00:00<?, ?it/s]

Найдено 5400 файлов, из них 583 с разметкой


  0%|          | 0/5394 [00:00<?, ?it/s]

Найдено 5394 файлов, из них 609 с разметкой
11310 11310
(6608, 19437)
Learning rate set to 0.137552
0:	learn: 0.6815293	test: 0.7919958	best: 0.7919958 (0)	total: 392ms	remaining: 38.8s
1:	learn: 0.6788420	test: 0.7705675	best: 0.7919958 (0)	total: 667ms	remaining: 32.7s
2:	learn: 0.6808245	test: 0.7705675	best: 0.7919958 (0)	total: 937ms	remaining: 30.3s
3:	learn: 0.6801226	test: 0.7657459	best: 0.7919958 (0)	total: 1.21s	remaining: 29.1s
4:	learn: 0.6833669	test: 0.7643805	best: 0.7919958 (0)	total: 1.48s	remaining: 28.2s
5:	learn: 0.6835092	test: 0.7642006	best: 0.7919958 (0)	total: 1.76s	remaining: 27.6s
6:	learn: 0.6868534	test: 0.7650000	best: 0.7919958 (0)	total: 2.03s	remaining: 27s
7:	learn: 0.6874058	test: 0.7646568	best: 0.7919958 (0)	total: 2.3s	remaining: 26.5s
8:	learn: 0.6876890	test: 0.7651305	best: 0.7919958 (0)	total: 2.58s	remaining: 26.1s
9:	learn: 0.6882906	test: 0.7644444	best: 0.7919958 (0)	total: 2.87s	remaining: 25.8s
10:	learn: 0.6889105	test: 0.7645752	best: 

  0%|          | 0/5400 [00:00<?, ?it/s]

...

Найдено 5394 файлов, из них 32 с разметкой
553 553
(117, 325)
Learning rate set to 0.050787
0:	learn: 0.1269841	test: 0.0000000	best: 0.0000000 (0)	total: 4.33ms	remaining: 429ms
1:	learn: 0.1269841	test: 0.0000000	best: 0.0000000 (0)	total: 8.06ms	remaining: 395ms
2:	learn: 0.5619835	test: 0.4814815	best: 0.4814815 (2)	total: 11.9ms	remaining: 383ms
3:	learn: 0.5826772	test: 0.4814815	best: 0.4814815 (2)	total: 15.5ms	remaining: 372ms
4:	learn: 0.3116883	test: 0.1012658	best: 0.4814815 (2)	total: 19.2ms	remaining: 364ms
5:	learn: 0.5873016	test: 0.4859813	best: 0.4859813 (5)	total: 21.9ms	remaining: 343ms
6:	learn: 0.3658537	test: 0.1012658	best: 0.4859813 (5)	total: 25.9ms	remaining: 344ms
7:	learn: 0.3809524	test: 0.1012658	best: 0.4859813 (5)	total: 29.6ms	remaining: 341ms
8:	learn: 0.3809524	test: 0.1012658	best: 0.4859813 (5)	total: 32.3ms	remaining: 327ms
9:	learn: 0.3809524	test: 0.1012658	best: 0.4859813 (5)	total: 35.9ms	remaining: 323ms
10:	learn: 0.5714286	test: 0.4081633	b

  0%|          | 0/5400 [00:00<?, ?it/s]

Найдено 5400 файлов, из них 3 с разметкой


  0%|          | 0/5394 [00:00<?, ?it/s]

Найдено 5394 файлов, из них 3 с разметкой
18 18
(12, 1)
Learning rate set to 0.028938
0:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 138us	remaining: 13.7ms
1:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 516us	remaining: 25.3ms
2:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 813us	remaining: 26.3ms
3:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 1.12ms	remaining: 26.9ms
4:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 1.41ms	remaining: 26.8ms
5:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 1.7ms	remaining: 26.7ms
6:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 1.98ms	remaining: 26.3ms
7:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 2.25ms	remaining: 25.9ms
8:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 2.54ms	remaining: 25.7ms
9:	learn: 0.6666667	test: 0.8571429	best: 0.8571429 (0)	total: 2.82ms	remaining: 25.4ms
10:	learn: 0.6666667	test: 0.8571429	b

In [ ]:
# повторяем то же самое для выбранных факторов
# сохраняем модели

best_fac = ['4_1', '3_9', '3_5', '4_2', '4_3']

models = {}

for corr_fac in best_fac:
    file_names = glob.glob('DataSet_Razmetra/**/**/**/**/Edition_Text.docx')

    parags = []
    labels = []

    highlighted_cnt = 0

    for f_name in tqdm(file_names):
        if f_name.split('/')[2]==corr_fac:
            try:
                doc = Document(f_name)

                # проверяем, что текст размечен
                for paragraph in doc.paragraphs:
                    is_highlighted_text = False

                    for run in paragraph.runs:
                        if not (run.font.highlight_color == None):
                            is_highlighted_text = True
                            break

                    if is_highlighted_text:
                        break

                # если текст размечен - разбираем на абзацы
                if is_highlighted_text:
                    highlighted_cnt = highlighted_cnt + 1

                    for paragraph in doc.paragraphs:
                        if paragraph.text != '': 
                            is_highlighted_parag = False
                            for run in paragraph.runs:
                                if not (run.font.highlight_color == None):
                                    is_highlighted_parag = True
                                    break
                            if is_highlighted_parag:
                                parags.append(paragraph.text)
                                labels.append(f_name.split('/')[2])

                # docs.append('\n'.join([p.text for p in doc.paragraphs]))
                # labels.append(file_names[i].split('/')[2])
            except:
                pass

    print('Найдено', len(file_names), 'файлов, из них', highlighted_cnt, 'с разметкой')

    file_names = glob.glob('DataSet_Razmetra/**/**/**/**/NC_Edition_Text.docx')

    highlighted_cnt = 0

    for f_name in tqdm(file_names):
        if f_name.split('/')[2]==corr_fac:
            try:
                doc = Document(f_name)

                # проверяем, что текст размечен
                for paragraph in doc.paragraphs:
                    is_highlighted_text = False

                    for run in paragraph.runs:
                        if not (run.font.highlight_color == None):
                            is_highlighted_text = True
                            break

                    if is_highlighted_text:
                        break

                # если текст размечен - разбираем на абзацы
                if is_highlighted_text:
                    highlighted_cnt = highlighted_cnt + 1

                    for paragraph in doc.paragraphs:
                        if paragraph.text != '': 
                            is_highlighted_parag = False
                            for run in paragraph.runs:
                                if not (run.font.highlight_color == None):
                                    is_highlighted_parag = True
                                    break
                            if is_highlighted_parag:
                                parags.append(paragraph.text)
                                labels.append('None')

                # docs.append('\n'.join([p.text for p in doc.paragraphs]))
                # labels.append(file_names[i].split('/')[2])
            except:
                pass

    print('Найдено', len(file_names), 'файлов, из них', highlighted_cnt, 'с разметкой')
    print(len(parags), len(labels))
    
    parags_1 = parags[:len(parags)//2]
    parags_2 = parags[len(parags)//2:]
    labels_1 = labels[:len(labels)//2]
    labels_2 = labels[len(labels)//2:]

    train_x = parags_1[:int(0.75*len(parags_1))] + parags_2[:int(0.75*len(parags_2))]
    train_y = labels_1[:int(0.75*len(labels_1))] + labels_2[:int(0.75*len(labels_2))]
    test_x = parags_1[int(0.75*len(parags_1)):] + parags_2[int(0.75*len(parags_2)):]
    test_y = labels_1[int(0.75*len(labels_1)):] + labels_2[int(0.75*len(labels_2)):]
    len(train_x), len(train_y), len(test_x), len(test_y)

    data_train = pd.DataFrame({'parag': train_x,
                              'label': train_y}).drop_duplicates()

    data_test = pd.DataFrame({'parag': test_x,
                              'label': test_y}).drop_duplicates()

    vectorizer = TfidfVectorizer(min_df=5, max_df=100000, ngram_range=(1,5))
    features_train = vectorizer.fit_transform(data_train.parag)
    print(features_train.shape)

    features_test = vectorizer.transform(data_test.parag)

    clf = CatBoostClassifier(n_estimators=100,
                            eval_metric='F1',)
    clf.fit(features_train, data_train.label, 
            eval_set=(features_test, data_test.label))
    y_test_pred = clf.predict(features_test)

    models[corr_fac] = (vectorizer, clf)

    print(corr_fac)
    print(classification_report(data_test.label, y_test_pred))

  0%|          | 0/5400 [00:00<?, ?it/s]

Найдено 5400 файлов, из них 881 с разметкой


  0%|          | 0/5394 [00:00<?, ?it/s]

Найдено 5394 файлов, из них 806 с разметкой
6719 6719
(4358, 12427)
Learning rate set to 0.124112
0:	learn: 0.1575809	test: 0.1200000	best: 0.1200000 (0)	total: 171ms	remaining: 16.9s
1:	learn: 0.1973356	test: 0.1838820	best: 0.1838820 (1)	total: 341ms	remaining: 16.7s
2:	learn: 0.2289206	test: 0.2060470	best: 0.2060470 (2)	total: 520ms	remaining: 16.8s
3:	learn: 0.2435897	test: 0.2078212	best: 0.2078212 (3)	total: 687ms	remaining: 16.5s
4:	learn: 0.2513047	test: 0.2259136	best: 0.2259136 (4)	total: 858ms	remaining: 16.3s
5:	learn: 0.2712535	test: 0.2259136	best: 0.2259136 (4)	total: 1.02s	remaining: 16.1s
6:	learn: 0.2992673	test: 0.2566845	best: 0.2566845 (6)	total: 1.2s	remaining: 15.9s
7:	learn: 0.3247734	test: 0.2661035	best: 0.2661035 (7)	total: 1.36s	remaining: 15.7s
8:	learn: 0.3409346	test: 0.2691903	best: 0.2691903 (8)	total: 1.54s	remaining: 15.6s
9:	learn: 0.3874420	test: 0.3024876	best: 0.3024876 (9)	total: 1.71s	remaining: 15.4s
10:	learn: 0.4122563	test: 0.3171206	best: 

  0%|          | 0/5400 [00:00<?, ?it/s]

...

Найдено 5394 файлов, из них 217 с разметкой
3433 3433
(1864, 6088)
Learning rate set to 0.100626
0:	learn: 0.1422594	test: 0.0609137	best: 0.0609137 (0)	total: 74.2ms	remaining: 7.35s
1:	learn: 0.1779141	test: 0.1250000	best: 0.1250000 (1)	total: 148ms	remaining: 7.28s
2:	learn: 0.2475634	test: 0.1266968	best: 0.1266968 (2)	total: 225ms	remaining: 7.27s
3:	learn: 0.2596899	test: 0.1261261	best: 0.1266968 (2)	total: 306ms	remaining: 7.35s
4:	learn: 0.2596899	test: 0.1345291	best: 0.1345291 (4)	total: 380ms	remaining: 7.22s
5:	learn: 0.3069767	test: 0.1603376	best: 0.1603376 (5)	total: 456ms	remaining: 7.14s
6:	learn: 0.3263061	test: 0.1428571	best: 0.1603376 (5)	total: 536ms	remaining: 7.12s
7:	learn: 0.3247706	test: 0.1434599	best: 0.1603376 (5)	total: 609ms	remaining: 7.01s
8:	learn: 0.3354603	test: 0.1440678	best: 0.1603376 (5)	total: 683ms	remaining: 6.91s
9:	learn: 0.3345521	test: 0.1434599	best: 0.1603376 (5)	total: 769ms	remaining: 6.92s
10:	learn: 0.3375912	test: 0.1355932	best:

  0%|          | 0/5400 [00:00<?, ?it/s]

Найдено 5400 файлов, из них 180 с разметкой


  0%|          | 0/5394 [00:00<?, ?it/s]

Найдено 5394 файлов, из них 175 с разметкой
756 756
(464, 1345)
Learning rate set to 0.071374
0:	learn: 0.2061069	test: 0.0000000	best: 0.0000000 (0)	total: 19.4ms	remaining: 1.92s
1:	learn: 0.2068966	test: 0.0000000	best: 0.0000000 (0)	total: 35.8ms	remaining: 1.75s
2:	learn: 0.2068966	test: 0.0000000	best: 0.0000000 (0)	total: 55.4ms	remaining: 1.79s
3:	learn: 0.2783883	test: 0.2178218	best: 0.2178218 (3)	total: 72ms	remaining: 1.73s
4:	learn: 0.2898551	test: 0.2178218	best: 0.2178218 (3)	total: 88ms	remaining: 1.67s
5:	learn: 0.3932203	test: 0.2857143	best: 0.2857143 (5)	total: 104ms	remaining: 1.63s
6:	learn: 0.4248366	test: 0.3018868	best: 0.3018868 (6)	total: 120ms	remaining: 1.6s
7:	learn: 0.4248366	test: 0.3018868	best: 0.3018868 (6)	total: 136ms	remaining: 1.57s
8:	learn: 0.4401294	test: 0.3018868	best: 0.3018868 (6)	total: 152ms	remaining: 1.54s
9:	learn: 0.4585987	test: 0.3148148	best: 0.3148148 (9)	total: 169ms	remaining: 1.52s
10:	learn: 0.4634921	test: 0.3148148	best: 0.3

In [ ]:
import pickle

with open('models.pickle', 'wb') as f:
    pickle.dump(models, f)

In [ ]:
import pickle

# models - словарь из кортежей (vectorizer, cat)
with open('models.pickle', 'rb') as f:
    models = pickle.load(f)

best_fac = ['3_9', '3_5', '4_3']
TRESHOLD = 0.75

f_name = 'DataSet_Razmetra/Республика Коми/3_7/A631C9EF-FA79-4453-BAF2-129A55551695/Edition_7/NC_Edition_Text.docx'
doc = Document(f_name)

for paragraph in doc.paragraphs:    

    indicator = False
    # проверяем абзац модельками
    for fac in best_fac:  
        par = paragraph.text
        # пропущена предоброботка текста, как на обучении - с утра попробую поправить
        vectorizer, clf = models[fac]
        if clf.predict_proba(vectorizer.transform([par]))[0][1]>TRESHOLD:
            indicator = True
            break
    # красим текст
    if indicator:
        print(paragraph.text)

5. Настоящий Приказ вступает в силу в установленном порядке, за исключением положений Административного регламента, касающихся межведомственного информационного взаимодействия, которые вступают в силу не ранее 1 июля 2012 года.
28. Заявителю по его желанию предоставляется возможность предварительной записи для представления документов на получение государственной услуги. Предварительная запись может осуществляться как при личном обращении заявителя в центр по предоставлению государственных услуг или МФЦ, так и по телефону.
99. Жалоба может быть направлена через организацию почтовой связи, иную организацию, осуществляющую доставку корреспонденции, через МФЦ, с использованием информационно-телекоммуникационной сети "Интернет", Портала государственных и муниципальных услуг (функций) Республики Коми, Единого портала государственных и муниципальных услуг (функций) (по рекомендуемой форме согласно приложению N 12 к настоящему Административному регламенту), а также может быть принята при личн